In [1]:
import pandas as pd
import numpy as np
import os
import re
from typing import List

# CÁC HÀM XỬ LÝ CHÍNH

# 1. Hàm chuẩn hóa giá gốc (Original_Price) và giá sau khi giảm (Sales_Price)

In [2]:
def normalize_price_string_logic(price):
    """
    Hàm chuẩn hóa giá, xử lý tất cả giá trị dưới dạng chuỗi (object).
    Phân biệt giữa dấu '.' hàng nghìn và dấu '.' thập phân.
    """
    # 1. Chuyển giá trị đầu vào thành dạng chuỗi để xử lý
    price_str = str(price)

    # 2. Nếu là chuỗi rỗng hoặc 'nan', trả về giá trị rỗng
    if price_str.lower() == 'nan' or not price_str:
        return np.nan
        
    # 3. Xóa các ký tự chữ đi (ví dụ 'Từ ')
    # Giữ lại số và dấu chấm để xử lý ở bước sau
    cleaned_str = re.sub(r'[^\d.]', '', price_str)
    
    if not cleaned_str:
        return np.nan

    # 4. Kiểm tra xem dấu '.' là hàng nghìn hay thập phân
    try:
        # TH1: Dấu '.' là ngăn cách hàng nghìn (vd: '450.000', '1.250.000')
        # Quy tắc: Có dấu chấm VÀ phần cuối sau dấu chấm có 3 ký tự
        if '.' in cleaned_str and len(cleaned_str.split('.')[-1]) == 3:
            # Xóa tất cả dấu chấm và chuyển thành số
            final_price = int(cleaned_str.replace('.', ''))
            return final_price
            
        # TH2: Dấu '.' là thập phân (vd: '480.0') hoặc không có dấu '.'
        else:
            # Chuyển thành số thực (float), sau đó nhân 1000
            # Việc chuyển thành float sẽ xử lý được cả '480.0' và '280000'
            numeric_val = float(cleaned_str)
            if numeric_val < 1000:
                return int(numeric_val * 1000)
            else:
                return int(numeric_val)

    except (ValueError, IndexError):
        # Nếu có bất kỳ lỗi nào trong quá trình chuyển đổi, trả về rỗng
        return np.nan

# 2. Load và gộp dữ liệu

In [3]:
def load_and_combine_data(file_paths: List[str], column_map: dict) -> pd.DataFrame:
    """
    Đọc nhiều file CSV, chuẩn hóa tên cột cho từng file, loại bỏ cột trùng lặp 
    được tạo ra sau khi chuẩn hóa, rồi gộp chúng lại.
    """
    df_list = []
    for path in file_paths:
        # Bước 1: Đọc từng file
        df_temp = pd.read_csv(path)
        
        # Bước 2: Áp dụng đổi tên cột trước
        df_temp = df_temp.rename(columns=column_map)
        
        # Bước 3: Xóa các cột bị trùng tên (được tạo ra từ Bước 2)
        # Đây là thứ tự đúng để sửa lỗi
        df_temp = df_temp.loc[:, ~df_temp.columns.duplicated(keep='first')]
        
        # Thêm DataFrame đã sạch vào danh sách
        df_list.append(df_temp)
        
    print(f"Đã đọc, chuẩn hóa và dọn dẹp cột cho {len(df_list)} file.")
    
    # Gộp các DataFrame đã hoàn toàn sạch sẽ lại
    return pd.concat(df_list, ignore_index=True)

# 3. Chuẩn hóa tên cột

In [4]:
def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Chuẩn hóa tên các cột và loại bỏ các cột có tên trùng lặp sau khi đổi tên.
    """
    column_mapping = {
        'bus_company': 'bus_name',
        'bus_star': 'bus_rating',
        'bus_type': 'seat_type',
        'duration_h_m': 'duartion',
        'drop_off_point': 'drop_of_point',
        'arrival_date': 'departure_date', 
        'price_original': 'original_price',
        'price_discount': 'sale_price',
        'discount_percent': 'percent_discount',
    }
    
    # Bước 1: Đổi tên các cột theo mapping
    df = df.rename(columns=column_mapping)
    
    # Bước 2: Xóa các cột bị trùng tên, chỉ giữ lại cột xuất hiện đầu tiên
    # Đây là dòng code quan trọng để sửa lỗi
    df = df.loc[:, ~df.columns.duplicated(keep='first')]
    
    print("Đã chuẩn hóa và loại bỏ các cột có tên trùng lặp.")
    return df

# 4. Áp dụng hàm chuẩn hóa giá cả

In [5]:
def clean_price_columns(df: pd.DataFrame) -> pd.DataFrame:
    # Áp dụng hàm mới, tạo cột 'price_cleaned' và 'sale_price_cleaned'
    df['price_cleaned'] = df['original_price'].apply(normalize_price_string_logic)
    df['sale_price_cleaned'] = df['sale_price'].apply(normalize_price_string_logic)
    # In kết quả ra để so sánh
    print("Kết quả sau khi làm sạch :")
    print(df[['original_price', 'price_cleaned']].head(-10))
    print(df['price_cleaned'].dtype)
    print(df[['sale_price', 'sale_price_cleaned']].head(-10))
    print(df['sale_price_cleaned'].dtype)
    # Ghi đè cột
    df['original_price'] = df['price_cleaned']
    df['sale_price'] = df['sale_price_cleaned']
    print("Đã ghi đè cột 'original_price' bằng dữ liệu từ 'price_cleaned'")
    print("Đã ghi đè cột 'sale_price' bằng dữ liệu từ 'sale_price_cleaned'")
    # Xóa cột 'price_cleaned' và 'sale_price_cleaned' không còn cần thiết
    df = df.drop('price_cleaned', axis=1)
    df = df.drop('sale_price_cleaned', axis=1)
    print("Đã xóa cột 'price_cleaned'")
    print("Đã xóa cột 'sale_price_cleaned'")

    return df


# 5. Hàm xử lí giá trị bị thiếu

In [6]:
def handle_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Xử lý các giá trị thiếu: xóa cột không cần thiết và điền giá trị còn lại.
    """
    # 1. Xóa các cột có quá nhiều giá trị thiếu hoặc không cần thiết
    cols_to_drop = [col for col in df.columns if 'Unnamed' in col]
    # Xóa cột arrival_date vì nó bị trùng lặp thông tin hoặc không đồng nhất
    if 'arrival_date' in df.columns:
        cols_to_drop.append('arrival_date')
        
    df = df.drop(columns=cols_to_drop, errors='ignore')
    print(f"Đã xóa các cột không cần thiết: {cols_to_drop}")

    # 2. Điền giá trị thiếu cho các cột rating
    rating_cols = ['An toàn', 'Thông tin chính xác', 'Thông tin đầy đủ',
                   'Thái độ nhân viên', 'Tiện nghi & thoải mái',
                   'Chất lượng dịch vụ', 'Đúng giờ']
    
    # Chỉ xử lý các cột rating tồn tại trong df
    existing_rating_cols = [col for col in rating_cols if col in df.columns]
    df[existing_rating_cols] = df.groupby('bus_name')[existing_rating_cols].transform(lambda x: x.ffill().bfill())
    
    condition = df['bus_rating'].astype(str) == '0 (0)'
    
    # Đếm số hàng sẽ bị thay đổi
    rows_to_change = df[condition].shape[0]
    # Kiểm tra xem có cột bus_rating nào bằng 0 không, nếu có thì chuyển tất cả các cột rating bằng 0
    if rows_to_change > 0:
        print(f"Tìm thấy {rows_to_change} hàng có rating bằng 0. Đang cập nhật...")
        # Bước 3: Dùng .loc để chọn các hàng và cột thỏa mãn rồi gán giá trị 0
        df.loc[condition, existing_rating_cols] = 0
    else:
        print("Không tìm thấy hàng nào có rating bằng 0 để cập nhật.")
        
    # Nếu sau đó vẫn còn NaN (do cả nhà xe không có rating nào), fill bằng median hoặc 0
    df[existing_rating_cols] = df[existing_rating_cols].fillna(df[existing_rating_cols].median())
    print("Đã điền giá trị thiếu cho các cột rating.")

    # 3. Điền giá trị thiếu cho các cột dạng object
    object_cols = df.select_dtypes(include=['object']).columns
    df[object_cols] = df[object_cols].fillna('')
    print("Đã điền giá trị thiếu cho các cột dạng chuỗi.")

    # 4. Điền giá trị bị thiếu cho 563 đòng đầu tiên cột start_point và destination
    df.loc[:562, 'start_point'] = "Sài Gòn"
    df.loc[:562, 'destination'] = "Nha Trang"
    print("Đã thêm dữ liệu 'Sài Gòn' và 'Nha Trang' vào 563 dòng đầu tiên (cột start_point và destination).")
    
    return df

def fill_missing_prices(df: pd.DataFrame) -> pd.DataFrame:
    """
    Điền các giá trị giá (original_price, sale_price) bị thiếu
    """
    print("====================================")
    print("--- Bắt đầu xử lý giá bị thiếu ---")
    print(f"Số original_price thiếu ban đầu: {df['original_price'].isnull().sum()}")
    print(f"Số sale_price thiếu ban đầu: {df['sale_price'].isnull().sum()}")

    # 1. Nếu original_price thiếu, lấy giá trị từ sale_price (nếu có)
    df['original_price'] = df['original_price'].fillna(df['sale_price'])
    
    # 2. Điền các giá trị còn thiếu của original_price bằng TRUNG VỊ theo nhóm
    df['original_price'] = df.groupby(['bus_name', 'seat_type'])['original_price'].transform(lambda x: x.fillna(x.median()))
    
    # Xử lý trường hợp có cả một nhóm không có giá nào -> điền bằng trung vị toàn bộ cột
    if df['original_price'].isnull().any():
        df['original_price'] = df['original_price'].fillna(df['original_price'].median())

    # 3. Điền các giá trị còn thiếu của sale_price bằng giá trị từ original_price đã được làm sạch
    df['sale_price'] = df['sale_price'].fillna(df['original_price'])
    
    print("\n--- Xử lý hoàn tất ---")
    print(f"Số original_price còn thiếu: {df['original_price'].isnull().sum()}")
    print(f"Số sale_price còn thiếu: {df['sale_price'].isnull().sum()}")
    
    return df

def clean_and_fill_discount(df: pd.DataFrame) -> pd.DataFrame:
    """
    Chuẩn hóa cột percent_discount về dạng số và điền các giá trị bị thiếu.
    """
    print("--- Bắt đầu chuẩn hóa và điền cột percent_discount ---")

    # 1. Chuẩn hóa cột về dạng số.
    # Xóa ký tự '%' và chuyển cột thành dạng số. Những giá trị không hợp lệ sẽ thành NaN.
    df['percent_discount'] = df['percent_discount'].astype(str).str.replace('%', '', regex=False)
    df['percent_discount'] = pd.to_numeric(df['percent_discount'], errors='coerce')
    
    print(f"Số percent_discount thiếu sau khi chuẩn hóa: {df['percent_discount'].isnull().sum()}")

    # 2. Tính toán và điền các giá trị còn thiếu.
    # Điều kiện: original_price > 0 và percent_discount đang bị thiếu
    condition = (df['original_price'] > 0) & (df['percent_discount'].isnull())

    # Tính toán phần trăm giảm giá từ các cột giá
    discount = ((df.loc[condition, 'sale_price'] - df.loc[condition, 'original_price']) / df.loc[condition, 'original_price']) * 100

    # Gán giá trị đã tính (làm tròn) vào những chỗ còn trống
    df.loc[condition, 'percent_discount'] = np.round(discount)
    
    # 3. Nếu vẫn còn NaN (ví dụ do giá gốc = 0), điền 0
    df['percent_discount'] = df['percent_discount'].fillna(0)
    
    # 4. Đảm bảo toàn bộ cột là kiểu số nguyên (integer) cho đồng nhất
    df['percent_discount'] = df['percent_discount'].astype(int)
    
    print("\n--- Xử lý hoàn tất ---")
    print(f"Số percent_discount còn thiếu: {df['percent_discount'].isnull().sum()}")
    print(f"Kiểu dữ liệu của cột sau khi xử lý: {df['percent_discount'].dtype}")
    print("===================================================================")
    
    return df

# 6. Tách cột 'bus_rating' thành hai cột 'bus_rating_score' và 'bus_rating_count'.

In [7]:
def split_bus_rating_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Tách cột 'bus_rating' từ dạng chuỗi "điểm (số lượt)" thành hai cột số riêng biệt
    là 'bus_rating_score' và 'bus_rating_count'.
    """
    print("--- Bắt đầu tách cột bus_rating ---")
    
    # Chuyển cột về dạng chuỗi để xử lý an toàn
    rating_str = df['bus_rating'].astype(str)

    # 1. Dùng regex để trích xuất điểm rating (phần số ở đầu)
    df['bus_rating_score'] = pd.to_numeric(rating_str.str.extract(r'(\d+\.?\d*)')[0], errors='coerce')

    # 2. Dùng regex để trích xuất số lượt đánh giá (phần số trong ngoặc)
    df['bus_rating_count'] = pd.to_numeric(rating_str.str.extract(r'\((\d+)\)')[0], errors='coerce')

    # 3. Điền giá trị 0 cho những trường hợp không trích xuất được (nếu có)
    df['bus_rating_score'] = df['bus_rating_score'].fillna(0)
    df['bus_rating_count'] = df['bus_rating_count'].fillna(0)
    
    # 4. Đảm bảo cột số lượt đánh giá là kiểu số nguyên
    df['bus_rating_count'] = df['bus_rating_count'].astype(int)
    
    print("--- Tách cột hoàn tất ---")
    print("Đã tạo hai cột mới: 'bus_rating_score' và 'bus_rating_count'.")
    print(f"Kiểu dữ liệu của bus_rating_score: {df['bus_rating_score'].dtype}")
    print(f"Kiểu dữ liệu của bus_rating_count: {df['bus_rating_count'].dtype}")
    print("===========================================================")

    return df

# 7. Hàm lưu DataFrame đã làm sạch vào CSV

In [8]:
def save_cleaned_data(df: pd.DataFrame, output_path: str):
    """
    Lưu DataFrame đã làm sạch vào file CSV.
    """
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Đã tạo thư mục: {output_dir}")

    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"File đã được lưu thành công tại: {output_path}")

# HÀM THỰC THI

In [9]:
def run_cleaning_pipeline(file_paths: List[str], output_path: str) -> pd.DataFrame:
    """
    Thực thi toàn bộ quy trình làm sạch dữ liệu.
    """
    print("===== BẮT ĐẦU QUY TRÌNH LÀM SẠCH DỮ LIỆU =====")
    
    # Định nghĩa column mapping (hàm chuẩn hóa tên cột)
    column_mapping = {
        'bus_company': 'bus_name',
        'bus_star': 'bus_rating',
        'bus_type': 'seat_type',
        'duration_h_m': 'duration',
        'drop_off_point': 'drop_of_point',
        'arrival_date': 'departure_date', 
        'price_original': 'original_price',
        'price_discount': 'sale_price',
        'discount_percent': 'percent_discount',
    }
    
    # 1. Đọc và gộp dữ liệu (với hàm đã được cải tiến)
    df = load_and_combine_data(file_paths, column_map=column_mapping)
    
    # 2. Chuẩn hóa tên cột (giờ chỉ còn nhiệm vụ xóa cột trùng lặp nếu có)
    df = standardize_columns(df)
    
    # 3. Xóa dữ liệu trùng lặp
    initial_rows = len(df)
    df = df.drop_duplicates().reset_index(drop=True)
    print(f"Đã xóa {initial_rows - len(df)} dòng trùng lặp.")
    
    # 4. Làm sạch cột giá
    df = clean_price_columns(df)
    
    # 5. Xử lý giá trị thiếu
    df = handle_missing_values(df)

    # 6. Xử lý giá trị bị thiếu (original_price và sales_price)
    df = fill_missing_prices(df)

    # 7. Xử lý giá trị bị thiếu (percent_discount)
    df = clean_and_fill_discount(df)

    # 8. Tách cột bus_rating
    df = split_bus_rating_column(df)
    
    # 9. Lưu kết quả
    save_cleaned_data(df, output_path)
    
    print("===== KẾT THÚC QUY TRÌNH LÀM SẠCH DỮ LIỆU =====")
    print("===============================================")
    return df

In [10]:
if __name__ == '__main__':
    # Định nghĩa đường dẫn tới các file dữ liệu thô
    raw_data_dir = '../../data/raw'
    files_to_process = [
        os.path.join(raw_data_dir, 'sg_nt_11102025.csv'),
        os.path.join(raw_data_dir, 'hn_th_15102025.csv'),
        os.path.join(raw_data_dir, 'Sài Gòn-Gia Lai-17-10-2025.csv')
    ]
    
    # Định nghĩa đường dẫn file đầu ra
    processed_data_path = '../../data/processed/cleaned_bus_data.csv'
    
    # Chạy quy trình
    cleaned_df = run_cleaning_pipeline(
        file_paths=files_to_process,
        output_path=processed_data_path
    )
    
    # Hiển thị thông tin của DataFrame đã làm sạch
    print("\\n===== THÔNG TIN DỮ LIỆU SAU KHI LÀM SẠCH =====")
    cleaned_df.info()

print("Kiểm tra dữ liệu bị thiếu: ")
cleaned_df.isnull().sum()*100/len(cleaned_df)

===== BẮT ĐẦU QUY TRÌNH LÀM SẠCH DỮ LIỆU =====
Đã đọc, chuẩn hóa và dọn dẹp cột cho 3 file.
Đã chuẩn hóa và loại bỏ các cột có tên trùng lặp.
Đã xóa 2 dòng trùng lặp.
Kết quả sau khi làm sạch :
     original_price  price_cleaned
0           450.000       450000.0
1           350.000       350000.0
2           280.000       280000.0
3           299.000       299000.0
4           480.000       480000.0
...             ...            ...
1003          500.0       500000.0
1004          350.0       350000.0
1005          350.0       350000.0
1006          650.0       650000.0
1007          350.0       350000.0

[1008 rows x 2 columns]
float64
     sale_price  sale_price_cleaned
0       320.000            320000.0
1       300.000            300000.0
2       250.000            250000.0
3       199.000            199000.0
4       400.000            400000.0
...         ...                 ...
1003      450.0            450000.0
1004      300.0            300000.0
1005        NaN              

bus_name                 0.0
bus_rating               0.0
seat_type                0.0
departure_time           0.0
pick_up_point            0.0
duration                 0.0
arrival_time             0.0
drop_of_point            0.0
departure_date           0.0
original_price           0.0
sale_price               0.0
percent_discount         0.0
notification             0.0
An toàn                  0.0
Thông tin chính xác      0.0
Thông tin đầy đủ         0.0
Thái độ nhân viên        0.0
Tiện nghi & thoải mái    0.0
Chất lượng dịch vụ       0.0
Đúng giờ                 0.0
start_point              0.0
destination              0.0
bus_rating_score         0.0
bus_rating_count         0.0
dtype: float64

In [11]:
print("In ra 5 dòng đầu: ")
cleaned_df.head()

In ra 5 dòng đầu: 


,bus_name,bus_rating,seat_type,departure_time,pick_up_point,duration,arrival_time,drop_of_point,departure_date,original_price,...,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ,start_point,destination,bus_rating_score,bus_rating_count
0,Đà Lạt ơi,4.8 (3425),Limousine 24 Phòng ĐÔI,23:30,• Trạm Hàng Xanh,7h,06:30,• Trạm Nha Trang,(12/10),450000.0,...,4.8,4.8,4.8,4.8,4.8,4.8,Sài Gòn,Nha Trang,4.8,3425
1,Bình Minh Tải,4.7 (3650),Limousine 22 Phòng Đơn,22:30,• Văn Phòng Quận 1,7h5m,05:35,• Văn phòng Nha Trang,(12/10),350000.0,...,4.6,4.7,4.6,4.6,4.6,4.8,Sài Gòn,Nha Trang,4.7,3650
2,Huỳnh Gia,4.7 (8534),Giường nằm 38 chỗ (WC),22:30,• Văn Phòng Phạm Ngũ Lão,6h30m,05:00,• Văn Phòng Nha Trang,(12/10),280000.0,...,4.7,4.7,4.7,4.6,4.6,4.7,Sài Gòn,Nha Trang,4.7,8534
3,An Anh Limousine,4.8 (8322),Limousine 34 Phòng Đơn,23:30,• Văn Phòng Quận 5,6h30m,06:00,• Văn phòng Nha Trang,(12/10),299000.0,...,4.7,4.7,4.7,4.7,4.7,4.6,Sài Gòn,Nha Trang,4.8,8322
4,Khanh Phong,4.7 (16845),Limousine 20 giường phòng (WC),05:15,• Văn Phòng Phạm Ngũ Lão - Quận 1.,6h10m,11:25,• Văn Phòng Nha Trang (KS Mường Thanh),,480000.0,...,4.7,4.7,4.7,4.6,4.6,4.8,Sài Gòn,Nha Trang,4.7,16845
